## Convert JSON Attribute Descriptions to Attribute Objects

This script automatically parses attribute descriptions from JSON files and converts them to the `Attribute` class format.


In [20]:
import random
import json
from typing import List, Dict, Tuple, Any
from dataclasses import dataclass
from enum import Enum
import itertools


In [21]:
class AttributeType(Enum):
    """Attribute value types"""
    STRING = "str"
    INTEGER = "int"
    FLOAT = "float"
    BOOLEAN = "bool"


class AttributeUsage(Enum):
    """Specific usage of attribute in queries"""
    CATEGORICAL = "categorical"    # For GROUP BY, categorical filters
    NUMERICAL = "numerical"        # For COUNT, SUM, AVG, MIN, MAX, numerical comparisons
    GENERAL = "general"            # For all usage


class AttributeModality(Enum):
    """Source modality of the attribute"""
    TEXT = "text"           # Plain text data
    IMAGE = "image"         # Image data (e.g., X-ray, photos)
    TABLE = "table"         # Table data
    STRUCTURED = "structured"  # Structured data from database


@dataclass
class Attribute:
    """
    Represents a database attribute with rich metadata.
    
    Attributes:
        name: Name of the attribute (column name)
        table: Table this attribute belongs to
        value_type: Data type of the attribute values (str, int, float, bool)
        usage: How this attribute is typically used in queries
        modality: Source modality of the data (text, image, table, etc.)
        is_nullable: Whether this attribute can have NULL values
        description: Optional description of the attribute
    """
    name: str
    table: str
    value_type: AttributeType
    usage: AttributeUsage
    modality: AttributeModality
    is_nullable: bool = False
    description: str = ""
    
    @property
    def full_name(self) -> str:
        """Return fully qualified name: table.attribute"""
        return f"{self.table}.{self.name}"

    def attribute_name(self) -> str:
        """Return the name of the attribute"""
        return self.name
    
    def is_groupable(self) -> bool:
        """Check if attribute can be used in GROUP BY"""
        return self.usage in [AttributeUsage.CATEGORICAL]
    
    def is_aggregatable(self) -> bool:
        """Check if attribute can be used with aggregation functions"""
        return self.usage == AttributeUsage.NUMERICAL
    
    def is_joinable(self) -> bool:
        """Check if attribute can be used as join key"""
        return self.usage == AttributeUsage.IDENTIFIER
    
    def supports_comparison(self) -> bool:
        """Check if attribute supports comparison operators (<, >, =, etc.)"""
        return self.value_type in [AttributeType.INTEGER, AttributeType.FLOAT, AttributeType.STRING]
    
    def supports_like(self) -> bool:
        """Check if attribute supports LIKE operator"""
        return self.value_type == AttributeType.STRING



In [22]:
def parse_attribute_description(desc_line: str) -> Tuple[str, str]:
    """
    Parse a single attribute description line.
    
    Format: "attribute_name: description text"
    
    Returns:
        Tuple of (attribute_name, description)
    """
    if ':' not in desc_line:
        return desc_line.strip(), ""
    
    parts = desc_line.split(':', 1)
    attr_name = parts[0].strip()
    description = parts[1].strip() if len(parts) > 1 else ""
    return attr_name, description


def infer_attribute_type(attr_name: str, description: str) -> AttributeType:
    """
    Infer the AttributeType based on attribute name and description.
    
    Rules:
    - If description mentions 'integer', 'number of', 'count', or name ends with '_count'/'_num' -> INTEGER
    - If description mentions 'age', 'awards', or '0/1' pattern -> INTEGER  
    - If description mentions 'date', 'float', 'score' -> FLOAT (dates stored as timestamps)
    - If description mentions 'boolean', 'whether', '1 if yes, 0 if' -> BOOLEAN
    - Otherwise -> STRING
    """
    name_lower = attr_name.lower()
    desc_lower = description.lower()
    
    # Check for integer types
    integer_keywords = ['integer', 'number of', 'enter an integer', 'awards']
    integer_names = ['age', 'awards', 'teaching', 'count', 'num']
    
    for keyword in integer_keywords:
        if keyword in desc_lower:
            return AttributeType.INTEGER
    
    for name_part in integer_names:
        if name_part in name_lower:
            return AttributeType.INTEGER
    
    # Check for boolean types (stored as int 0/1)
    boolean_patterns = ['whether', '1 if yes', '1 if true', '0 if no', '0 if none']
    for pattern in boolean_patterns:
        if pattern in desc_lower:
            return AttributeType.INTEGER  # Boolean stored as 0/1
    
    # Check for date types (can be stored as string in specific format)
    if 'date' in name_lower or 'date in' in desc_lower:
        return AttributeType.STRING  # Dates stored as formatted strings
    
    # Default to STRING
    return AttributeType.STRING


def infer_attribute_usage(attr_name: str, description: str) -> AttributeUsage:
    """
    Infer the AttributeUsage based on attribute name and description.
    
    Rules:
    - If 'choose one from', 'select one from', 'choose only one' -> CATEGORICAL
    - If description has fixed choices like '[...]' with limited options -> CATEGORICAL
    - If mentions 'age', 'awards', 'number of', 'count' -> NUMERICAL
    - Otherwise -> GENERAL
    """
    name_lower = attr_name.lower()
    desc_lower = description.lower()
    
    # Check for categorical (has fixed choices)
    categorical_patterns = [
        'choose one from', 'select one from', 'choose only one',
        'nationality', 'zodiac', 'continent', 'marriage', 'field', 'genre'
    ]
    
    for pattern in categorical_patterns:
        if pattern in desc_lower or pattern in name_lower:
            return AttributeUsage.CATEGORICAL
    
    # Check for numerical
    numerical_patterns = ['age', 'awards', 'number of', 'count', 'teaching']
    for pattern in numerical_patterns:
        if pattern in name_lower or pattern in desc_lower:
            return AttributeUsage.NUMERICAL
    
    # Default to GENERAL
    return AttributeUsage.GENERAL


def convert_json_to_attributes(json_path: str, dataset_name: str = "Art") -> Dict[str, List[Attribute]]:
    """
    Convert a JSON attribute descriptions file to Attribute objects.
    
    Args:
        json_path: Path to the JSON file
        dataset_name: Name of the dataset (used as table name prefix)
    
    Returns:
        Dictionary mapping source table names to lists of Attribute objects
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    result = {}
    
    for source_key, descriptions in data.items():
        # Determine modality based on source key
        if 'Art' in source_key or 'Image' in source_key or 'Photo' in source_key:
            modality = AttributeModality.IMAGE
        elif 'Text' in source_key or 'Wiki' in source_key:
            modality = AttributeModality.TEXT
        elif 'Table' in source_key:
            modality = AttributeModality.TABLE
        else:
            modality = AttributeModality.TEXT  # Default
        
        # Create table name from source key
        table_name = f"{dataset_name}_{source_key}"
        
        attributes = []
        for desc_line in descriptions:
            attr_name, description = parse_attribute_description(desc_line)
            
            # Infer types
            value_type = infer_attribute_type(attr_name, description)
            usage = infer_attribute_usage(attr_name, description)
            
            # Check if nullable (mentions 'leave empty', 'if not applicable', 'optional')
            is_nullable = any(p in description.lower() for p in ['leave empty', 'if not applicable', 'optional', 'or empty'])
            
            attr = Attribute(
                name=attr_name,
                table=table_name,
                value_type=value_type,
                usage=usage,
                modality=modality,
                is_nullable=is_nullable,
                description=description
            )
            attributes.append(attr)
        
        result[source_key] = attributes
    
    return result


def print_attributes_summary(attributes_dict: Dict[str, List[Attribute]]):
    """Print a summary of converted attributes."""
    print("=" * 80)
    print("CONVERTED ATTRIBUTES SUMMARY")
    print("=" * 80)
    
    for source, attrs in attributes_dict.items():
        print(f"\n📋 Source: {source}")
        print(f"   Table: {attrs[0].table if attrs else 'N/A'}")
        print(f"   Modality: {attrs[0].modality.value if attrs else 'N/A'}")
        print(f"   Total attributes: {len(attrs)}")
        print("-" * 60)
        
        for attr in attrs:
            nullable_str = "?" if attr.is_nullable else ""
            print(f"   • {attr.name}{nullable_str}: {attr.value_type.value} ({attr.usage.value})")


# Convert Art_descriptions.json
json_path = "/data/dengqiyan/UDA-Bench/Query/Finan/Finan_descriptions.json"
attributes = convert_json_to_attributes(json_path, dataset_name="Finan")

# Print summary
print_attributes_summary(attributes)


CONVERTED ATTRIBUTES SUMMARY

📋 Source: finance
   Table: Finan_finance
   Modality: text
   Total attributes: 25
------------------------------------------------------------
   • company_name: str (general)
   • registered_office: str (general)
   • exchange_code: str (general)
   • principal_activities: str (general)
   • board_members: str (general)
   • executive_profiles: str (numerical)
   • revenue: str (general)
   • net_profit_or_loss: str (general)
   • total_Debt: str (general)
   • total_assets: str (general)
   • cash_reserves: str (general)
   • net_assets: str (general)
   • earnings_per_share: str (general)
   • dividend_per_share: str (general)
   • largest_shareholder: str (general)
   • the_highest_ownership_stake: str (numerical)
   • major_equity_changes: int (categorical)
   • major_events: str (general)
   • bussiness_sales: str (general)
   • bussiness_profit: str (general)
   • bussiness_cost: str (general)
   • business_segments_num: int (numerical)
   • busin

In [23]:
# Export attributes to a reusable format (optional: save to JSON)
def export_attributes_to_json(attributes_dict: Dict[str, List[Attribute]], output_path: str):
    """Export converted attributes to a JSON file for reuse."""
    export_data = {}
    
    for source, attrs in attributes_dict.items():
        export_data[source] = [
            {
                "name": attr.name,
                "table": attr.table,
                "value_type": attr.value_type.value,
                "usage": attr.usage.value,
                "modality": attr.modality.value,
                "is_nullable": attr.is_nullable,
                "description": attr.description
            }
            for attr in attrs
        ]
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Exported attributes to {output_path}")


# Get all attributes as a flat list for query generation
def get_all_attributes(attributes_dict: Dict[str, List[Attribute]]) -> List[Attribute]:
    """Flatten all attributes into a single list."""
    all_attrs = []
    for attrs in attributes_dict.values():
        all_attrs.extend(attrs)
    return all_attrs


# Get attributes by modality
def get_attributes_by_modality(attributes_dict: Dict[str, List[Attribute]], 
                                modality: AttributeModality) -> List[Attribute]:
    """Get all attributes with a specific modality."""
    result = []
    for attrs in attributes_dict.values():
        for attr in attrs:
            if attr.modality == modality:
                result.append(attr)
    return result


# Example: Get all image attributes from Art dataset
image_attrs = get_attributes_by_modality(attributes, AttributeModality.IMAGE)
text_attrs = get_attributes_by_modality(attributes, AttributeModality.TEXT)

print(f"\n📷 IMAGE attributes ({len(image_attrs)}):")
for attr in image_attrs:
    print(f"   • {attr.name}")

print(f"\n📝 TEXT attributes ({len(text_attrs)}):")
for attr in text_attrs:
    print(f"   • {attr.name}")

# Export to JSON (uncomment to save)
export_attributes_to_json(attributes, "/data/dengqiyan/UDA-Bench/Query/Finan/Finan_attributes.json")



📷 IMAGE attributes (0):

📝 TEXT attributes (25):
   • company_name
   • registered_office
   • exchange_code
   • principal_activities
   • board_members
   • executive_profiles
   • revenue
   • net_profit_or_loss
   • total_Debt
   • total_assets
   • cash_reserves
   • net_assets
   • earnings_per_share
   • dividend_per_share
   • largest_shareholder
   • the_highest_ownership_stake
   • major_equity_changes
   • major_events
   • bussiness_sales
   • bussiness_profit
   • bussiness_cost
   • business_segments_num
   • business_risks
   • remuneration_policy
   • auditor
✅ Exported attributes to /data/dengqiyan/UDA-Bench/Query/Finan/Finan_attributes.json
